In [1]:
import glob as glob
import os
import numpy as np
import mne 
import shutil
from mne.datasets import eegbci
from mne.datasets import sleep_physionet
import matplotlib.pyplot as plt
import pandas as pd
from mne.preprocessing import (ICA, corrmap, create_ecg_epochs,
                               create_eog_epochs)
import torch
import numpy as np
import platform, os, re, multiprocessing
import pyedflib
import time  # Import the time module
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import platform, os, re, multiprocessing
import pandas as pd
import pyedflib
import mne
import matplotlib.pyplot as plt
import time  # Import the time module
import os
import mne
import warnings
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.pipeline import make_pipeline
import glob as glob
import numpy as np
import mne
import shutil
from mne.datasets import eegbci
from mne.datasets import sleep_physionet
import pandas as pd
from mne.preprocessing import (ICA, corrmap, create_ecg_epochs,
                               create_eog_epochs)
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from datetime import timedelta
from mne.datasets import sample
from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA, FastICA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [2]:
import glob
import os

path = "/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/"
raw_list = []

for i in range(1,23):  # Adjust the range as needed based on the number of files you have
    pattern = f"rbd{i}.edf"
    file_path = os.path.join(path, pattern)
    if os.path.exists(file_path):
        raw_list.append(file_path)

print(raw_list)

['/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd1.edf', '/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd2.edf', '/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd3.edf', '/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd4.edf', '/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd5.edf', '/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd7.edf', '/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd8.edf', '/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd9.edf', '/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd10.edf', '/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd11.edf', '/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd12.edf', '/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd13.edf',

In [3]:
# Define the desired sampling frequency
desired_sfreq = 128

# Read each file and append to a list of Raw objects
raw_objects = []
for file in raw_list:
    raw = mne.io.read_raw_edf(file, preload=True, verbose=0, infer_types=True)
    raw_objects.append(raw)

# Resample each Raw object to the desired sampling frequency
for raw_obj in raw_objects:
    raw_obj.resample(desired_sfreq)

print(raw_objects)

[<RawEDF | rbd1.edf, 15 x 3874688 (30271.0 s), ~443.4 MB, data loaded>, <RawEDF | rbd2.edf, 20 x 4584960 (35820.0 s), ~699.6 MB, data loaded>, <RawEDF | rbd3.edf, 24 x 4646400 (36300.0 s), ~850.8 MB, data loaded>, <RawEDF | rbd4.edf, 18 x 6597248 (51541.0 s), ~906.0 MB, data loaded>, <RawEDF | rbd5.edf, 22 x 4454400 (34800.0 s), ~747.7 MB, data loaded>, <RawEDF | rbd7.edf, 23 x 4047488 (31621.0 s), ~710.3 MB, data loaded>, <RawEDF | rbd8.edf, 22 x 4124288 (32221.0 s), ~692.3 MB, data loaded>, <RawEDF | rbd9.edf, 20 x 4539008 (35461.0 s), ~692.6 MB, data loaded>, <RawEDF | rbd10.edf, 22 x 3294848 (25741.0 s), ~553.1 MB, data loaded>, <RawEDF | rbd11.edf, 22 x 3294848 (25741.0 s), ~553.1 MB, data loaded>, <RawEDF | rbd12.edf, 23 x 4120448 (32191.0 s), ~723.1 MB, data loaded>, <RawEDF | rbd13.edf, 23 x 4005120 (31290.0 s), ~702.8 MB, data loaded>, <RawEDF | rbd14.edf, 16 x 4285568 (33481.0 s), ~523.2 MB, data loaded>, <RawEDF | rbd15.edf, 12 x 6516608 (50911.0 s), ~596.6 MB, data loaded>,

In [4]:
path_csv = "/home/Duchuy220602/thesis/file_csv"
csv_list = []

for i in range(1, 23):  # Adjust the range as needed based on the number of files you have
    file_pattern = f"rbd{i}.csv"
    file_path = os.path.join(path_csv, file_pattern)
    if os.path.exists(file_path):
        csv_list.append(file_path)


for i in range(len(raw_objects)):
    raw = raw_objects[i]
    csv = csv_list[i]

    df = pd.read_csv(csv)

    # Convert the 'Time [hh:mm:ss]' column to a time format
    df['Time [hh:mm:ss]'] = pd.to_datetime(df['Time [hh:mm:ss]'], format='%H:%M:%S').dt.time

    # Extract the necessary columns from your DataFrame
    duration = df['Duration[s]']
    description = df['Sleep Stage']

    # Convert onset times to seconds
    onset = list(range(0, len(df)))  # Replace with your actual onset times

    # Create MNE Annotations
    my_annot = mne.Annotations(onset=onset, duration=duration, description=description)

    # Set annotations on the copied EEG data
    raw.set_annotations(my_annot)

    # Print the annotations
    print(raw.annotations)

<Annotations | 932 segments: MT (5), R (126), S1 (52), S2 (472), S3 (61), ...>
<Annotations | 1912 segments: R (182), S1 (24), S2 (788), S3 (390), S4 ...>
<Annotations | 1764 segments: R (127), S1 (22), S2 (911), S3 (260), S4 ...>
<Annotations | 1817 segments: R (293), S1 (17), S2 (862), S3 (395), S4 ...>
<Annotations | 1711 segments: R (165), S1 (3), S2 (863), S3 (239), S4 ...>
<Annotations | 1413 segments: R (66), S1 (35), S2 (596), S3 (374), S4 ...>
<Annotations | 1658 segments: MT (1), R (144), S1 (26), S2 (289), S3 ...>
<Annotations | 1448 segments: R (206), S1 (53), S2 (396), S3 (322), S4 ...>
<Annotations | 1071 segments: MT (5), R (81), S1 (160), S2 (258), S3 (45), ...>
<Annotations | 1067 segments: MT (5), R (81), S1 (168), S2 (242), S3 (45), ...>
<Annotations | 1433 segments: MT (5), R (160), S1 (24), S2 (801), S3 ...>
<Annotations | 1802 segments: R (200), S1 (10), S2 (1114), S3 (190), S4 ...>
<Annotations | 1537 segments: MT (2), R (296), S1 (162), S2 (525), S3 ...>
<Annota

In [5]:
# for i, raw in enumerate(raw_objects):
#     print(f"Plotting Raw object {i+1}...")
#     raw.plot()

In [6]:
# # Define the current and new channel names
# current_channel_names = ['C4A1']
# new_channel_names = ['C4-A1']

# # Iterate over the list of Raw objects
# for i in range(len(raw_objects)):
#     raw = raw_objects[i]
    
#     # Check if the current Raw object contains any of the channels to be renamed
#     if any(channel in raw.ch_names for channel in current_channel_names):
#         # Create a channel map that maps the old channel names to the new channel names
#         channel_map = {current_channel_names[i]: new_channel_names[i] for i in range(len(current_channel_names)) if current_channel_names[i] in raw.ch_names}

#         # Rename the channels in the Raw object
#         raw.rename_channels(channel_map)

#         # Print the new channel names
#         print(raw.ch_names)

In [7]:
# Define a dictionary mapping channel names to their types
channel_types = {
"Fp2-F4": "eeg",
"F4-C4": "eeg",
"C4-P4": "eeg",
"P4-O2": "eeg",
"C4-A1": "eeg", 
}

# Define the default type for EEG channels
default_type = 'stim'

# Apply the channel type mapping to all Raw objects
for i in range(len(raw_objects)):
    raw = raw_objects[i]
    # Get the channel names that are present in both the Raw object and the channel_types dictionary
    common_channel_names = set(raw.ch_names) & set(channel_types.keys())
    if not common_channel_names:
        print(f"No common channels found between the Raw object and the channel_types dictionary. Skipping...")
        continue

    # Set channel types for selected channels
    channel_types_dict = {channel_name: channel_types.get(channel_name, default_type) for channel_name in common_channel_names}
    raw.set_channel_types(channel_types_dict)

    # Select only the EEG channels
    eeg_channel_names = [channel_name for channel_name in common_channel_names if channel_types_dict[channel_name] == 'eeg']
    if eeg_channel_names:
        raw = raw.pick(eeg_channel_names)
    else:
        print(f"No EEG channels found in the Raw object. Skipping...")

In [8]:
for i, raw in enumerate(raw_objects):
    print(f"Raw object {i+1} has {raw.info['nchan']} channels.")

Raw object 1 has 5 channels.
Raw object 2 has 5 channels.
Raw object 3 has 5 channels.
Raw object 4 has 5 channels.
Raw object 5 has 5 channels.
Raw object 6 has 5 channels.
Raw object 7 has 5 channels.
Raw object 8 has 5 channels.
Raw object 9 has 5 channels.
Raw object 10 has 5 channels.
Raw object 11 has 5 channels.
Raw object 12 has 5 channels.
Raw object 13 has 5 channels.
Raw object 14 has 5 channels.
Raw object 15 has 5 channels.
Raw object 16 has 5 channels.
Raw object 17 has 5 channels.
Raw object 18 has 5 channels.
Raw object 19 has 5 channels.
Raw object 20 has 5 channels.
Raw object 21 has 5 channels.


In [9]:
for i, raw in enumerate(raw_objects):
    print(f"Raw object {i+1} ({raw.filenames[0]})")

Raw object 1 (/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd1.edf)
Raw object 2 (/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd2.edf)
Raw object 3 (/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd3.edf)
Raw object 4 (/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd4.edf)
Raw object 5 (/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd5.edf)
Raw object 6 (/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd7.edf)
Raw object 7 (/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd8.edf)
Raw object 8 (/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd9.edf)
Raw object 9 (/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd10.edf)
Raw object 10 (/mnt/data_lab513/data_thesis_duchuy/physionet.org/files/capslpdb/1.0.0/rbd11.edf)
Raw object 11 (/mnt/data_lab513/data_thesis_duc

In [10]:
# for i, raw in enumerate(raw_objects):
#     print(f"Plotting Raw object {i+1}...")
#     raw.plot()

In [11]:
# If you want to concatenate the Raw objects into a single object
raw = mne.concatenate_raws(raw_objects)

In [12]:
# Print the concatenated annotations
print(raw.annotations)

<Annotations | 31064 segments: BAD boundary (20), EDGE boundary (20), MT ...>


In [13]:
raw.save('/home/Duchuy220602/thesis/modify_file/rbd_concatenated_raw.fif', overwrite=True)

Writing /home/Duchuy220602/thesis/modify_file/rbd_concatenated_raw.fif
Closing /home/Duchuy220602/thesis/modify_file/rbd_concatenated_raw.fif
[done]
